In [0]:
%fs ls 's3://nokia-assignment-s3/assignment_data/raw_xml_data/'

In [0]:
SHOW CATALOGS;

In [0]:
CREATE CATALOG IF NOT EXISTS `nokia-assginment-catalog`
MANAGED LOCATION 's3://nokia-assignment-s3/'
COMMENT 'this is a catalog for nokia assginment';

In [0]:
SHOW CATALOGS;

In [0]:
select current_catalog();

In [0]:
USE CATALOG `nokia-assginment-catalog`;

In [0]:
select current_catalog();

In [0]:
CREATE SCHEMA IF NOT EXISTS assignment_data
MANAGED LOCATION 's3://nokia-assignment-s3/assignment_data/';

In [0]:
SHOW SCHEMAS;

In [0]:
CREATE SCHEMA IF NOT EXISTS bronze
MANAGED LOCATION 's3://nokia-assignment-s3/assignment_data/bronze/';

CREATE SCHEMA IF NOT EXISTS silver
MANAGED LOCATION 's3://nokia-assignment-s3/assignment_data/silver/';

CREATE SCHEMA IF NOT EXISTS gold
MANAGED LOCATION 's3://nokia-assignment-s3/assignment_data/gold/';

In [0]:
USE SCHEMA assignment_data;

In [0]:
CREATE EXTERNAL VOLUME xml_raw_data LOCATION 's3://nokia-assignment-s3/assignment_data/raw_xml_data/';


In [0]:
%fs ls '/Volumes/nokia-assginment-catalog/assignment_data/xml_raw_data'

In [0]:
CREATE EXTERNAL VOLUME bronze_volume LOCATION 's3://nokia-assignment-s3/assignment_data/bronze/';
